In [1]:
%load_ext autoreload
%autoreload 2

# Init

In [2]:
import json
import sys
import re
import pickle as pkl
from pathlib import Path
from functools import partial
from collections import defaultdict
sys.path.append("../src")

import pandas as pd
from matplotlib import pyplot as plt
from joblib import Parallel, delayed
from tqdm import tqdm

import constants
from scoring import score
from gen.util import read_data, write_jsonl

In [3]:
model_data_p = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/dumps")

pred_p = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/thesis/predictions")

fever_actual_p = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/fever-nei-sampled")
cfever_actual_p = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/climatefever-neg-sampled")
sf_actual_p = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/scifact-nei-sampled/all.n5.nei.jsonl")

actual_pls = list(fever_actual_p.glob("*.n5.nei.jsonl")) + list(cfever_actual_p.glob("*.n5.nei.jsonl")) + [sf_actual_p]
actual_pls = [p for p in actual_pls if "train" not in p.stem]

# Concatenate Evidences

In [4]:
res_d = {"fever": defaultdict(list), "climatefever": defaultdict(list), "scifact": defaultdict(list)}

for actual_dp in tqdm(actual_pls):
    dataset = actual_dp.parent.stem.split("-")[0]
    split = actual_dp.stem.split(".")[0]
    actual_data = read_data(actual_dp)
    if dataset == "climatefever":
        score_obj = partial(score.ClimateFEVERScorer, actual_data=actual_data, oracle_rte=False, oracle_ir=True, max_evidence=None)
    else:
        score_obj = partial(score.FEVERScorer, actual_data=actual_data, oracle_rte=False, oracle_ir=True, max_evidence=None)
    for pp in pred_p.joinpath("doc", dataset).glob(f"*.{split}.jsonl"):
        preds = read_data(pp)
        res_d[dataset][split].append(score_obj(prediction_data=preds, score_name=pp.stem))

100%|██████████| 5/5 [00:07<00:00,  1.53s/it]


In [8]:
for score in res_d["fever"]["dev"]:
    print(score._score_name)
    print(score.classification_report)
    print("===")

fever-da.dev
                 precision    recall  f1-score   support

NOT ENOUGH INFO       0.79      0.84      0.82      3333
        REFUTES       0.90      0.39      0.55      3333
       SUPPORTS       0.57      0.86      0.69      3333

       accuracy                           0.70      9999
      macro avg       0.76      0.70      0.68      9999
   weighted avg       0.76      0.70      0.68      9999

===
fever-climatefever-bert-base-uncased.dev
                 precision    recall  f1-score   support

NOT ENOUGH INFO       1.00      1.00      1.00      3333
        REFUTES       0.96      0.90      0.93      3333
       SUPPORTS       0.90      0.96      0.93      3333

       accuracy                           0.95      9999
      macro avg       0.95      0.95      0.95      9999
   weighted avg       0.95      0.95      0.95      9999

===
fever-xlnet-base-cased.dev
                 precision    recall  f1-score   support

NOT ENOUGH INFO       1.00      0.99      1.00   

In [9]:
for score in res_d["climatefever"]["dev"]:
    print(score._score_name)
    print(score.classification_report)
    print("===")

fever-da.dev
                 precision    recall  f1-score   support

NOT ENOUGH INFO       0.41      0.21      0.28        95
        REFUTES       0.40      0.31      0.35        51
       SUPPORTS       0.51      0.73      0.60       132

       accuracy                           0.48       278
      macro avg       0.44      0.42      0.41       278
   weighted avg       0.46      0.48      0.45       278

===
fever-climatefever-bert-base-uncased.dev
                 precision    recall  f1-score   support

NOT ENOUGH INFO       0.84      0.77      0.80        95
        REFUTES       0.48      0.63      0.54        51
       SUPPORTS       0.77      0.73      0.75       132

       accuracy                           0.72       278
      macro avg       0.70      0.71      0.70       278
   weighted avg       0.74      0.72      0.73       278

===
fever-xlnet-base-cased.dev
                 precision    recall  f1-score   support

NOT ENOUGH INFO       0.71      0.21      0.33   

In [5]:
for score in res_d["scifact"]["all"]:
    print(score._score_name)
    print(score.classification_report)
    print("===")

fever-climatefever-xlnet-base-cased.all
                 precision    recall  f1-score   support

NOT ENOUGH INFO       0.90      1.00      0.95       416
        REFUTES       0.75      0.37      0.50       237
       SUPPORTS       0.76      0.89      0.82       456

       accuracy                           0.82      1109
      macro avg       0.81      0.75      0.75      1109
   weighted avg       0.81      0.82      0.80      1109

===
climatefever-xlnet-base-cased.all
                 precision    recall  f1-score   support

NOT ENOUGH INFO       0.18      0.02      0.04       416
        REFUTES       0.39      0.38      0.39       237
       SUPPORTS       0.47      0.84      0.60       456

       accuracy                           0.44      1109
      macro avg       0.35      0.42      0.34      1109
   weighted avg       0.34      0.44      0.35      1109

===
fever-climatefever-da.all
                 precision    recall  f1-score   support

NOT ENOUGH INFO       0.48    